In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.image import imread

In [2]:
my_data_dir = 'C:\\Users\\Matija\\Desktop\\seceno' #directory for preprocessed images (image processing will be explained)

In [3]:
test_path = my_data_dir+'\\test'
train_path = my_data_dir+'\\train'

In [4]:
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])  #turning pictures to grayscale

In [6]:
#using defined function to convert pictures
#for i in os.listdir(train_path+"\\SMA-"):
    #img_matrix = imread(train_path+"\\SMA-\\"+i)
    #img_siva = rgb2gray(img_matrix)
    #cv2.imwrite(os.path.join("C:\\Users\\Matija\\Desktop\\crnobelo\\train\\SMA-" , i),img_siva)

In [7]:
#cutting pictures, just catching area of potential SMN1 peak
#for i in os.listdir(train_path+"\\SMA+"):
   # img_matrix = imread(train_path+"\\SMA+\\"+i)
   # img_secena = img_matrix[165:470 ,270:340]
    #cv2.imwrite(os.path.join("C:\\Users\\Matija\\Desktop\\seceno\\train\\SMA+",i),img_secena)

In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator #importing ImageDataGenerator

In [9]:
image_gen = ImageDataGenerator(rescale=1/255, # Rescale the image by normalzing it.
                               fill_mode='nearest' # Fill in missing pixels with the nearest filled value
                              )

In [15]:
image_gen.flow_from_directory(train_path) #checking ImageDataGenerator
image_gen.flow_from_directory(test_path)

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D #importing model type and layers

In [ ]:
model = Sequential()

model.add(Conv2D(filters=1, kernel_size=(5,5), activation='relu',padding="same",strides=(1,1)))
model.add(Conv2D(filters=1, kernel_size=(5,5), activation='relu',padding="same",strides=(1,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=1, kernel_size=(3,3), activation='relu',padding="same",strides=(1,1)))
model.add(Conv2D(filters=1, kernel_size=(3,3), activation='relu',padding="same",strides=(1,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(1024))
model.add(Dropout(0.3))

model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=["accuracy"])

In [13]:
from tensorflow.keras.callbacks import EarlyStopping #importing and creating earlystopping
early_stop = EarlyStopping(monitor='val_loss',patience=2)

In [ ]:
batch_size = 2 #defining batch_size for training

In [ ]:
#crating image generator for training images
train_image_gen = image_gen.flow_from_directory(train_path,
                                                color_mode='grayscale',
                                               batch_size=batch_size,
                                               class_mode='binary',
                                               target_size=(305,70))

In [ ]:
#crating image generator for test images
test_image_gen = image_gen.flow_from_directory(test_path,
                                               color_mode='grayscale',
                                               batch_size=batch_size,
                                               class_mode='binary',
                                              target_size=(305,70),
                                              shuffle=False)

In [ ]:
train_image_gen.class_indices #checking which label corresponds with which category

In [ ]:
#fitting the model
results = model.fit_generator(train_image_gen,epochs=8,validation_data=test_image_gen,callbacks=[early_stop])

In [ ]:
#getting the metrics
metrics = pd.DataFrame(model.history.history)

In [ ]:
metrics[["loss","val_loss"]].plot() #plotting losses

In [ ]:
metrics[["accuracy","val_accuracy"]].plot() #plotting accuracies

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
print(classification_report(test_image_gen.classes,predictions))

In [ ]:
plt.figure(figsize=(3,3))
sns.heatmap(confusion_matrix(test_image_gen.classes,predictions),annot=True)